# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
import os
from google.colab import drive
from google.colab import files

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
clustering_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset_BMLP/hasil_clustering.csv')
clustering_df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Income_Binned,Cluster_DBSCAN,PCA_Cluster_DBSCAN,Cluster_KMeans,MntTotal
0,5524,1957,2,4,58138.0,0,0,2012-09-04,58,635.0,...,0,0,3,11,1,1,-1,0,1,25
1,2174,1954,2,4,46344.0,1,1,2014-03-08,38,11.0,...,0,0,3,11,0,1,-1,0,0,6
2,4141,1965,2,5,71613.0,0,0,2013-08-21,26,426.0,...,0,0,3,11,0,1,-1,0,1,21
3,6182,1984,2,5,26646.0,1,0,2014-02-10,26,11.0,...,0,0,3,11,0,1,-1,0,0,8
4,5324,1981,4,3,58293.0,1,0,2014-01-19,94,173.0,...,0,0,3,11,0,1,-1,0,1,19


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [11]:
# Menentukan kolom target
target_column = 'Cluster_KMeans'

# Memisahkan fitur (X) dan label (y)
y = clustering_df[target_column]
X = clustering_df.drop(columns=[
    "Dt_Customer", "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5",
    "AcceptedCmp1", "AcceptedCmp2", "Complain", "Z_CostContact",
    "Z_Revenue", "Response", "ID", target_column, "Cluster_DBSCAN", "PCA_Cluster_DBSCAN"
])

# Membagi dataset menjadi data latih (80%) dan data uji (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Ukuran data latih:", X_train.shape, y_train.shape)
print("Ukuran data uji:", X_test.shape, y_test.shape)

Ukuran data latih: (1792, 20) (1792,)
Ukuran data uji: (448, 20) (448,)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [12]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Random Forest adalah algoritma machine learning yang digunakan untuk masalah klasifikasi dan regresi. Algoritma ini bekerja dengan cara membuat sejumlah pohon keputusan (decision trees) secara acak dan kemudian menggabungkan hasil dari masing-masing pohon untuk membuat keputusan akhir. Setiap pohon dibangun berdasarkan subset acak dari data pelatihan dan subset acak dari fitur yang digunakan pada setiap pemisahan dalam pohon. Hasil akhirnya adalah mayoritas prediksi (untuk klasifikasi) atau rata-rata (untuk regresi) dari semua pohon keputusan yang ada. Menurut saya, algoritma ini cocok dengan dataset yang saya gunakan karena saya menggunakan data dengan banyak fitur dan data yang besar serta noisy.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [13]:
y_pred = rf_model.predict(X_test)
print("Akurasi sebelum tuning:", accuracy_score(y_test, y_pred))
print("Laporan klasifikasi sebelum tuning:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Akurasi sebelum tuning: 0.9620535714285714
Laporan klasifikasi sebelum tuning:
               precision    recall  f1-score   support

           0       0.95      0.97      0.96       229
           1       0.97      0.95      0.96       219

    accuracy                           0.96       448
   macro avg       0.96      0.96      0.96       448
weighted avg       0.96      0.96      0.96       448

Confusion Matrix:
 [[223   6]
 [ 11 208]]


Hasil pelatihan menunjukkan akurasi dan F1-Score yang cukup mengesankan pada tahap awal pelatihan.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [14]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

cv_strategy = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=cv_strategy,  # Gunakan StratifiedKFold
    scoring='accuracy',
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=2, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [15]:
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [16]:
best_rf_model = grid_search.best_estimator_
y_pred_tuned = best_rf_model.predict(X_test)
print("Akurasi setelah tuning:", accuracy_score(y_test, y_pred_tuned))
print("Laporan klasifikasi setelah tuning:\n", classification_report(y_test, y_pred_tuned))
print("Confusion Matrix setelah tuning:\n", confusion_matrix(y_test, y_pred_tuned))

Akurasi setelah tuning: 0.9642857142857143
Laporan klasifikasi setelah tuning:
               precision    recall  f1-score   support

           0       0.96      0.97      0.97       229
           1       0.97      0.95      0.96       219

    accuracy                           0.96       448
   macro avg       0.96      0.96      0.96       448
weighted avg       0.96      0.96      0.96       448

Confusion Matrix setelah tuning:
 [[223   6]
 [ 10 209]]


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

In [17]:
print("Perbandingan hasil evaluasi:")
print("- Akurasi sebelum tuning:", accuracy_score(y_test, y_pred))
print("- Akurasi setelah tuning:", accuracy_score(y_test, y_pred_tuned))
if accuracy_score(y_test, y_pred_tuned) > accuracy_score(y_test, y_pred):
    print("Model meningkat setelah tuning.")
else:
    print("Model tidak meningkat setelah tuning. Coba parameter lain.")

Perbandingan hasil evaluasi:
- Akurasi sebelum tuning: 0.9620535714285714
- Akurasi setelah tuning: 0.9642857142857143
Model meningkat setelah tuning.


1. Perbandingan Hasil Evaluasi:
- Akurasi meningkat dari 0.9576 menjadi 0.9643 setelah tuning.
- Precision dan recall untuk kedua kelas tetap hampir sama, dengan sedikit peningkatan di kelas 0.
- Kesalahan klasifikasi menurun dari 19 kesalahan (8+11) menjadi 16 kesalahan (6+10).

2. Identifikasi Kelemahan Model:
- Precision dan recall untuk kelas 1 sedikit lebih rendah dibandingkan kelas 0, tetapi perbedaannya kecil.
- Perubahan akurasi kecil mengindikasikan bahwa model tidak mengalami overfitting atau underfitting secara signifikan.

3. Rekomendasi Tindakan Lanjutan:
- Evaluasi lanjutan dengan dataset yang lebih besar untuk memastikan model lain juga bekerja dengan baik.
- Mencoba metode balancing data jika perbedaan recall antar kelas menjadi lebih besar di dataset baru.
- Eksperimen dengan algoritma lain atau feature engineering tambahan jika peningkatan yang diinginkan belum tercapai.